# EWC

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training import EWC
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 100
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/ewc.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'ewc_lambda': 50.0
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    
    loggers=[InteractiveLogger()]
)

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [10]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = EWC(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [11]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/100 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:05<00:00, 85.94it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:05<00:00, 85.74it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:05<00:00, 86.19it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:05<00:00, 87.14it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:05<00:00, 84.00it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 95.73it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9081
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 97.29it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1631
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 97.44it/s]
> Eval on

  1%|          | 1/100 [01:57<3:14:25, 117.84s/it]


> Eval on experience 99 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp099 = 0.1115
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = 0.0000
	StreamForgetting/eval_phase/test_stream = 0.0000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1235
-- >> Start of training phase << --
100%|██████████| 469/469 [00:06<00:00, 73.02it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:06<00:00, 73.22it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:06<00:00, 76.12it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:06<00:00, 77.61it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:05<00:00, 78.17it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 101.61it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0096
	ExperienceForgetting/eval_phase/test_stream

  2%|▏         | 2/100 [03:56<3:13:38, 118.55s/it]

...


> Eval on experience 99 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp099 = 0.0980
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.6172
	StreamForgetting/eval_phase/test_stream = 0.6172
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.0980
-- >> Start of training phase << --
100%|██████████| 469/469 [00:39<00:00, 11.82it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:39<00:00, 11.82it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:40<00:00, 11.59it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:40<00:00, 11.72it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:39<00:00, 11.78it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 121.34it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.8101
	ExperienceForgetting/eval_phase/test_strea

 85%|████████▌ | 85/100 [4:22:45<1:06:27, 265.81s/it]


> Eval on experience 99 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp099 = 0.0980
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.6098
	StreamForgetting/eval_phase/test_stream = 0.6098
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.0980
-- >> Start of training phase << --
100%|██████████| 469/469 [00:40<00:00, 11.63it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:41<00:00, 11.42it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:40<00:00, 11.52it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:40<00:00, 11.64it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:41<00:00, 11.22it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 126.17it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.8101
	ExperienceForgetting/eval_phase/test_strea

 86%|████████▌ | 86/100 [4:27:19<1:02:36, 268.29s/it]


> Eval on experience 99 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp099 = 0.0980
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.6026
	StreamForgetting/eval_phase/test_stream = 0.6026
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.0980
-- >> Start of training phase << --
100%|██████████| 469/469 [00:41<00:00, 11.38it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:40<00:00, 11.55it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:38<00:00, 12.09it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:39<00:00, 11.88it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:39<00:00, 11.98it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 125.23it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.8101
	ExperienceForgetting/eval_phase/test_strea

# Get metrics

In [ ]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

# Store metrics

In [ ]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))